In [1]:
# true if working from tbi, false is working from home
tbi = True

if tbi:
    base_folder = '/scr/aldea/kgutenbrunner/github/MA/'
else:
    base_folder = '/Users/katringutenbrunner/Desktop/MA/github/MA/'


In [2]:

import os
import csv
import re

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from Bio import AlignIO, SeqIO


import RNA

In [ ]:
# get all the stk files except YFV
stk_folder = os.path.join(base_folder, 'data/tbfv_stk/')
skip = ['TYUV', 'XiFV', 'SREV', 'MPFV']
# SREV and TYUV have no beta stem
# XiFV has very small stems
stk_files = [
    os.path.join(stk_folder, file)
    for file in os.listdir(stk_folder)
    if file.endswith('xrRNA1.stk') and not any(skip_item in file for skip_item in skip)
]

In [4]:
# get the seqeuences from each stk file
sequences = []
rfs = []
for stk_file in stk_files:
    align = AlignIO.read(stk_file, "stockholm")
    rfs.append(align.column_annotations['reference_annotation'])
    for record in align:
        sequences.append(str(record.seq))



In [6]:
# create and fasta file from all the sequences

output_fa_CM = os.path.join(base_folder, 'cov_model/TBFV_xrRNA1/data/combined.fa')
with open(output_fa_CM, 'w') as f:
    for stk_file in stk_files:
        align = AlignIO.read(stk_file, "stockholm")
        for record in align:
            
            f.write(f'>{record.id}\n')
            f.write(f'{record.seq}\n')
            f.write(f"{align.column_annotations['secondary_structure']} #S\n")

In [7]:
# use mlocarna to create a alignment file
!mlocarna $output_fa_CM

zsh:1: command not found: mlocarna


In [ ]:
# save the output consensus secondary structure from mlocarna
alifold = '.........(.(((((..((.((((......(((.........))).((((............)))).)))).)))))))..)...........((....)).......'

In [ ]:
# create a stockholm file based on the mlocarna alignment

# get the alignment file
input_file = os.path.join(base_folder, 'cov_model/TBFV_xrRNA1/data/combined.out/results/result.aln')
# output name for the stockholm file
align_stk_file = os.path.join(base_folder, 'cov_model/TBFV_xrRNA1/data/alignment.stk')

input_file = os.path.join(base_folder, 'cov_model/TBFV_xrRNA1/data/combined.out/results/result.aln')


## Load the Clustal alignment
alignment = AlignIO.read(input_file, "clustal")
## add the secondary structure from mlocarna manually
alignment.column_annotations["secondary_structure_cons"] = alifold
alignment.column_annotations['secondary_structure'] = alifold

## Write the alignment in Stockholm format
AlignIO.write(alignment, align_stk_file, "stockholm")

In [ ]:
# create the covariance model based on the stockholm we just created
!cmbuild /scr/aldea/kgutenbrunner/working/cov_model/TBFV_xrRNA1/TBFV_xrRNA1.cm $stk_file

In [5]:
# load the just create model
cm_model = os.path.join(base_folder, 'cov_model/TBFV_xrRNA1/TBFV_xrRNA1.cm')
designs_file_CM = os.path.join(base_folder, 'cov_model/TBFV_xrRNA1/data/designed_sequences.out')

In [ ]:
# create 100 sequences from the model
# !cmemit -N 100 -a -o $designs_file_CM $cm_model